**This code below demonstrates how AWS Bedrock and Kendra can assist in quickly searching through a large volume of documents within seconds:**

Step 1: Install the neccessary libraries using ``pip install -r requirements.txt`` in the terminal

Step 2: Import libraries, setup aws credentials and client initialization

In [ ]:
import boto3
from langchain_aws import ChatBedrock, AmazonKendraRetriever
from langchain.prompts import PromptTemplate

# Setting up credentials
aws_access_key_id = ""
aws_secret_access_key = ""
aws_session_token = ""
kendra_index_id = ""
kendra_data_source_id = ""

# Initialize AWS Bedrock and Kendra client
bedrock_client = boto3.client('bedrock-runtime',region_name='us-east-1', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key, aws_session_token=aws_session_token)
kendra_client = boto3.client('kendra', region_name='us-east-1', aws_access_key_id=aws_access_key_id,aws_secret_access_key=aws_secret_access_key, aws_session_token=aws_session_token)

Step 3: Configure AWS Bedrock and Kendra

In [ ]:
# Configure LLM - AWS Bedrock - Anthropic Claude 3.0 Haiku
claude_3_client = ChatBedrock(
    client = bedrock_client,
    model_id = "anthropic.claude-3-haiku-20240307-v1:0",
    model_kwargs = { 
        "max_tokens": 500,
        "temperature": 0.0,
    }
)

# Configure AWS Kendra
kendra_retriever = AmazonKendraRetriever(index_id = kendra_index_id, top_k = 3, client = kendra_client, min_score_confidence = 0.5,
    # Attribute filter is applied to filter by language language, data source or other attributes
    attribute_filter={
        'AndAllFilters': [
            {
                "EqualsTo": {
                    "Key": "_language_code",
                    "Value": {"StringValue": "en"}
                }
            },
            {
                "EqualsTo": {
                    "Key": "_data_source_id",
                    "Value": {"StringValue": kendra_data_source_id}
                }
            }
        ]
    } 
)

Step 4: Invoke AWS Bedrock with prompt template to obtain clear and friendly response for the retrieved content from AWS Kendra.

In [ ]:
# Define prompt template for the Bedrock LLM when data is retrieved from Kendra
kendra_prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""You need to act as a world class document retrieval expert. I have tasks for you.\
    Given to you is the {context} of the document retreived from AWS Kendra. \
    Your answer should be related to the {question} provided. \
    Instead of saying based on the provided document, you should say I have found something based on the existing database. \
    You need to explaint clearly in point form (1, 2, n), for the document name and excerpt (Less than 50 words) for it. \
    Please merge the excerpt based on document name. """
)

# Define fallback prompt template for Bedrock LLM when no Kendra data is available
fallback_prompt_template = PromptTemplate(
    input_variables=["question"],
    template="""
    No relevant information was found in the database.
    
    Please provide a general answer to the question:
    {question}
    """
)

# Create LLMChain with Bedrock LLM for cases when Kendra data is available
llm_chain_with_kendra = kendra_prompt_template | claude_3_client

# Create LLMChain with Bedrock LLM for cases when Kendra data is not available
llm_chain_fallback = fallback_prompt_template | claude_3_client

def ask_question(question):
    # Retrieve context from Kendra and output the result
    context = kendra_retriever.invoke(question)
    print(context)

    if context: # If Kendra returns results
        # Prepare input dictionary with context and question
        input_data = {
            "context": context,  
            "question": question 
        }
        # Ask Bedrock LLM to generate an answer using the Kendra results
        response = llm_chain_with_kendra.invoke(input=input_data)
    else:  # If no relevant results from Kendra
        # Prepare input dictionary with only the question
        input_data = {
            "question": question
        }
        # Ask Bedrock LLM to generate a general answer
        response = llm_chain_fallback.invoke(input=input_data)
    return response

# Put your question here:
question = "What is the content stated in PDPA 2010 in Malaysia?"
answer = ask_question(question)

Step 5: Print the result

In [ ]:
# Result for your answer
print(answer.content)